In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 9.6 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 6.6 MB 50.7 MB/s 


In [2]:
!pip install -q datasets==2.0.0

     |████████████████████████████████| 325 kB 7.5 MB/s 
     |████████████████████████████████| 212 kB 65.1 MB/s 
     |████████████████████████████████| 1.1 MB 62.4 MB/s 
     |████████████████████████████████| 140 kB 71.8 MB/s 
     |████████████████████████████████| 127 kB 75.5 MB/s 
     |████████████████████████████████| 144 kB 71.7 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 271 kB 71.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!pip install -q seqeval


     |████████████████████████████████| 43 kB 813 kB/s 


In [4]:
!pip install -q pyyaml==5.1
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install -q torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

#!git clone https://github.com/facebookresearch/detectron2.git
#!python -m pip install -e detectron2

     |████████████████████████████████| 274 kB 6.9 MB/s 
     |████████████████████████████████| 763.5 MB 15 kB/s 
     |████████████████████████████████| 17.3 MB 483 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.8.0+cu101 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0+cu101 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-xszo9784
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-xszo9784
     |████████████████████████████████| 50 kB 3.7 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 
     |████████████████████████████████| 15

In [10]:
#imports
import os
import glob
import json
import random
from pathlib import Path
from difflib import SequenceMatcher



import cv2
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from IPython.display import display
import matplotlib
from PIL import Image
from matplotlib import pyplot, patches
from transformers import  LayoutLMv2Processor, LayoutXLMTokenizer, LayoutXLMProcessor, LayoutLMv3Processor
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

from datasets import load_dataset,Dataset

In [ ]:
import torch
torch.cuda.empty_cache()

In [29]:



#datasets = load_dataset("darentang/sroie",cache_dir=None)
#datasets = load_dataset("nielsr/funsd",cache_dir=None)
datasets = load_dataset("nielsr/funsd-layoutlmv3")

##creating a dataloader

from torch.utils.data import DataLoader

# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=2)


Reusing dataset funsd (/root/.cache/huggingface/datasets/nielsr___funsd/funsd/1.0.0/0e3f4efdfd59aa1c3b4952c517894f7b1fc4d75c12ef01bcc8626a69e41c1bb9)


  0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
#labels
labels = datasets['train'].features['ner_tags'].feature.names
print(labels)
#labels
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
label2id


['O', 'B-HEADER', 'I-HEADER', 'B-QUESTION', 'I-QUESTION', 'B-ANSWER', 'I-ANSWER']


{'B-ANSWER': 5,
 'B-HEADER': 1,
 'B-QUESTION': 3,
 'I-ANSWER': 6,
 'I-HEADER': 2,
 'I-QUESTION': 4,
 'O': 0}

#Text: The original word text.

Lemma: The base form of the word.

POS: The simple UPOS part-of-speech tag.

Tag: The detailed part-of-speech tag.

Dep: Syntactic dependency, i.e. the relation between tokens.

Shape: The word shape – capitalization, punctuation, digits.

is alpha: Is the token an alpha character?

is stop: Is the token part of a stop list, i.e. the most common words of the language?

In [37]:
datasets["train"]

Dataset({
    features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
    num_rows: 149
})

In [13]:
#processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr =False)

In [65]:
def read_image_from_dataset(img_data_str: str): 
    
    return np.array(img_data_str)

read_image_from_dataset(datasets["train"]["image"][0])


array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [114]:

## preprocess the dataset
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

# we need to define custom features
features = Features({
    'pixel_values': Array3D(dtype="float64", shape=( 3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
})

def preprocess_data(examples):
  """
  """
 
  images = [np.array(path.convert("RGB")) for path in examples['image']]
  words = examples['tokens']
  boxes = examples['bboxes']
  word_labels = examples['ner_tags']
  
  encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,
                             padding="max_length", truncation=True)
  
  return encoded_inputs

# single = datasets["train"][0:5]

# r = processor(single["image"],single["tokens"], boxes = single["bboxes"],word_labels=single["ner_tags"],  truncation=True)
# print(r.keys())


train_dataset = datasets["train"].map(preprocess_data, batched=True, remove_columns=datasets["train"].column_names, features=features)
test_dataset = datasets["test"].map(preprocess_data, batched=True, remove_columns=datasets["test"].column_names,features=features)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [115]:
train_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

In [116]:

##creating a dataloader

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2)

In [ ]:
!sudo apt-get install git-lfs

In [117]:
train_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

##creating a dataloader

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2)



from transformers import LayoutLMv3ForTokenClassification, TrainingArguments, Trainer
from datasets import load_metric
import numpy as np

model = LayoutLMv3ForTokenClassification.from_pretrained('microsoft/layoutlmv3-base',
                                                                      num_labels=len(label2id))

# Set id2label and label2id 
model.config.id2label = id2label
model.config.label2id = label2id

# Metrics
metric = load_metric("seqeval")
return_entity_level_metrics = True



Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [118]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

class FunsdTrainer(Trainer):
    def get_train_dataloader(self):
      return train_dataloader

    def get_test_dataloader(self, test_dataset):
      return test_dataloader


In [119]:
args = TrainingArguments(
    output_dir="layoutlmv3-finetuned-funsd-v2", # name of directory to store the checkpoints
    max_steps=1000, # we train for a maximum of 1,000 batches
    warmup_ratio=0.1, # we warmup a bit
    fp16=True # we use mixed precision (less memory consumption)
    , # after training, we'd like to push our model to the hub
     # this is the name we'll use for our model on the hub
)

# Initialize our Trainer
trainer = FunsdTrainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics,
)



max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend


In [120]:
trainer.train()

predictions, labels, metrics = trainer.predict(test_dataset)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 149
  Num Epochs = 27
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:755: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning


Step,Training Loss
500,0.415100
1000,0.011900


Saving model checkpoint to layoutlmv3-finetuned-funsd-v2/checkpoint-500
Configuration saved in layoutlmv3-finetuned-funsd-v2/checkpoint-500/config.json
Model weights saved in layoutlmv3-finetuned-funsd-v2/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:755: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning
Saving model checkpoint to layoutlmv3-finetuned-funsd-v2/checkpoint-1000
Configuration saved in layoutlmv3-finetuned-funsd-v2/checkpoint-1000/config.json
Model weights saved in layoutlmv3-finetuned-funsd-v2/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 50
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:755: FutureWarning: The `devic

In [121]:
metrics

{'test_ANSWER_f1': 0.9161134580567291,
 'test_ANSWER_number': 817,
 'test_ANSWER_precision': 0.9035714285714286,
 'test_ANSWER_recall': 0.9290085679314566,
 'test_HEADER_f1': 0.6266094420600858,
 'test_HEADER_number': 119,
 'test_HEADER_precision': 0.6403508771929824,
 'test_HEADER_recall': 0.6134453781512605,
 'test_QUESTION_f1': 0.9170506912442395,
 'test_QUESTION_number': 1077,
 'test_QUESTION_precision': 0.9103385178408051,
 'test_QUESTION_recall': 0.9238625812441968,
 'test_loss': 1.6439099311828613,
 'test_overall_accuracy': 0.8319267799833591,
 'test_overall_f1': 0.9,
 'test_overall_precision': 0.8925256472887152,
 'test_overall_recall': 0.9076005961251863,
 'test_runtime': 3.7891,
 'test_samples_per_second': 13.196,
 'test_steps_per_second': 1.847}

In [ ]:
## with autocorrection

metrics

{'test_ANSWER_f1': 0.8143283582089552,
 'test_ANSWER_number': 809,
 'test_ANSWER_precision': 0.7875288683602771,
 'test_ANSWER_recall': 0.8430160692212608,
 'test_HEADER_f1': 0.6133333333333333,
 'test_HEADER_number': 119,
 'test_HEADER_precision': 0.6509433962264151,
 'test_HEADER_recall': 0.5798319327731093,
 'test_QUESTION_f1': 0.860475968268782,
 'test_QUESTION_number': 1065,
 'test_QUESTION_precision': 0.8552875695732839,
 'test_QUESTION_recall': 0.8657276995305164,
 'test_loss': 0.770158052444458,
 'test_overall_accuracy': 0.8206864347420033,
 'test_overall_f1': 0.8276032649023002,
 'test_overall_precision': 0.8160975609756097,
 'test_overall_recall': 0.8394380331159057,
 'test_runtime': 3.3503,
 'test_samples_per_second': 14.924,
 'test_steps_per_second': 2.089}